### Explanation
This code analyses time lapse images of plates with growing colonies and gets their size and mean fluorescence over time together with some valuable characterization of their dynamics. It uses the python module FluoPi to perform these tasks.

##### Requirements
 - All images have to be same size


## Analysis

### Loading packages
As part of the FluoPi project we provide a python module to manage and analyse time lapse data. We will also use some common modules for plotting and analysing data:

In [ ]:
# Import the FluoPi module
import fluopi

# import some useful packages to manage the data

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

#modify some matplotlib parameters to manage the images for illustrator
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

### Loading Data

Specifying the folder name where the images are stored, we can count the files:

In [ ]:
fpath = '../Lab_27_1_17_pLac'
imCount = fluopi.count_files(fpath, 'jpg')

In [ ]:
fpath = 'D:\Dropbox\FTI\Raspiscope\Time lapse\Lab_27_1_17_pLac'
imCount = fluopi.count_files(fpath, 'jpg')

The FluoPi module provides a function to read images and save their values in 3-dimensional arrays (one for each channel). Now we can use this function to read and store the data of the images on a defined array for each channel:

In [ ]:
import os
fname = os.path.join(fpath, 'image_%04d.jpg')

Frames=12   # use one image every 12 frames

DataS={}
DataS['R'],DataS['G'],DataS['B'] = fluopi.get_im_data(Frames,imCount,fname)
DataS['Im']=fname     # to store the related image source

# also define a vector with the channels
channels=['R','G','B']

Finally is important know the time step value used on the timelapse. It can be know from the python timelapse code stored on the folder (named 'interval' on the code).

In [ ]:
#define it on hour units
dT= 1.0/12.0  #hours

#in this example interval = 5 min for each frame


With this, we can construct a **time vector** 'T' for *DataS*

In [ ]:
w,h,st=DataS['R'].shape
T=np.zeros((st))
for i in range(0,st):
    T[i]=(i)*Frames*dT

print(T)

### Data management

#### Background computation and substraction

To take the valuable data of images is good to know the amount of background we have on it, if it changes over time or how noisy is it.

By ploting a transect of pixel values for each channel on an image, we can figure the efect of background: 

In [ ]:
fluopi.row_transect(DataS, row=int(w/2), xframes=10)

As you can see, data shows "peaks" (colonies) and a basal value along the line (background signal)

Then, to distinguish cell fluorescence from media and background light, we can select some empty sector on the image (i.e. without colonies) and compute his mean value on each frame.

In [ ]:
BG = fluopi.BG_Val(512, 610, 280, 450, DataS, imCount)

We can substract this background value from each frame using BG_subst:

In [ ]:
DataSB = fluopi.BG_subst(DataS,BG)

and we can check the effect on the background

In [ ]:
_ = fluopi.BG_Val(512, 610, 280, 450, DataSB, imCount)

Background was practically eliminated.

### Get colony position
Here we are interested on identify the place where colonies are situated

A good way to do it is by sum the value of each pixel for all the channels and all the frames

In [ ]:
SumData = fluopi.dataOT(DataSB)

Now we can see each colony as a more clear point

#### Filter application
To improve the border detection and reduce the noise is recommended  to apply a filter to smooth the data. A simple and effective gaussian filter is provided by skimage package:
[skimage gaussian filter](http://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian)

In [ ]:
from skimage.filters import gaussian

The effect of the filter can be understood with the next example:

In [ ]:
#see the effect of the filter
a = np.zeros((3, 3))
a[1,1]=1
print(a)
gaussian(a, sigma=1)

Then we apply a the filter to the data (re-run the next line with different sigma values until you are agree with the smooth):

In [ ]:
sDatS,sDatSall,sDatST=fluopi.smoothDat(DataSB,sigma=0.7)

And we can see the plot of the sumarized result

In [ ]:
plt.imshow(sDatSall)
plt.colorbar()
plt.title('All channels')

#### Blob detection
Scikit-image provides several functions to detect Gaussian-like features ([blobs](http://scikit-image.org/docs/dev/auto_examples/plot_blob.html)). We apply one of these ([blob_log](http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_log)) to the smoothed sum of intensity, and it returns a list of position (x,y) and width (standard deviation):

To properly identification, it's important to set the threshold for each dataset manually (based on the previous colormap)

In [ ]:
blobD = fluopi.colonyBlob(sDatSall,0.26,fname%599,filename='Plots/DetectedBlobs')

With the previous procedure we have identified where each colony is placed

To perform further analysis on each colony it is necesary to store these regions of interest (ROI) and his data, with size given by the Gaussian width (x2 = 2 standard deviations).

In [ ]:
Rois,RoisC,NC = fluopi.obtain_rois(sDatST,blobD)

we can ilustrate the data with the first blob (colony)

In [ ]:
# show the first column of the first ROI
print(Rois['R'][0][:,1,1])
print(RoisC['R'][0][:,1,1]) #it contains only the pixels inside the circular region, and make the other equal to zero

The ROI let us to track every colony in a seperatelly way. The module includes a package to save an image of a selected ROI at selected times. Then is possible make timelapse grow of an indivial colony.

In [ ]:
idx=1
fname=('ROIs/Col'+str(idx)+'_ROI_step%d.png')
Times=[24,29,34,39,44,49]

fluopi.TL_ROI(Rois,idx,Times,fname,[2,3])

### Time dynamics of colonies
As we have estimated colony location (and maximmum size), we can analyse the dynamics of fluorescence and growth of them. 

#### Fluorescence dynamics
With the data obtained so far we can plot the fluorescence dynamics of colonies. We can choose some representative colonies and use the FluoPi rois_plt_Fdynam() function to perform this task:

In [ ]:
#define the selected colonies
Cols=[1,17,45,79,66,125,81,109,121]

#use the function to plot
fluopi.rois_plt_Fdynam(RoisC,T,Cols,filename='Plots/Channel_total_intensity')
print('Each line correspond to a different ROI')

#### Colony radius growth
We have the location and a final size estimate based on the total fluorescence of each colony over the time-lapse. We would like to know the size of the colony in each frame; from this we can compute the growth rate.

One way to do this is to find Gaussian blobs in each of the image regions of the colony, for each frame of the time-lapse.This is very slow

First we perform the sum of all channels for each frame to get the best representation of colony size:

In [ ]:
ACRois=fluopi.channelSum(RoisC,Cols) #circular ROIS

In [ ]:
ACRoisS=fluopi.channelSum(Rois,Cols) #square ROIS

The following loop finds blobs and stores the width in each frame. The result is a dict containing the size at each time point for every colony

In [ ]:
import time
start = time.time()
R_frame5=fluopi.frame_colony_radius(ACRoisS,Cols,0.37)
end = time.time()
print(str(end - start)+' seg')

Since the colony is roughly circular and flat, an estimate of volume is $R^2$. Lets plot this for some colonies. Plotting the log is useful because the slope is the relative growth rate. We can see the colonies slow their growth during the experiment.

In [ ]:
fluopi.plot_radius(R_frame5, Cols, T)
print('Each line correspond to one colony')

#### Check the radius estimate
To see how well the colony size is estimated, we can compare to the profile of a colony in a slice of its image region. The radius (red line) should give the border of the colony.

In [ ]:
IDCol=Cols[2]
plt.figure()
plt.imshow(ACRoisS[IDCol][:,:,-1], interpolation='none', cmap='gray')

fluopi.checkR(R_frame5[IDCol],ACRoisS,IDCol,T,filename='Plots/Chemograph_'+str(IDCol))


##### Getting the colony area over time $A(t)$
with the radio value we are able to get the colony area over time by taking it as a perfect circle: $$A(t) = \pi \cdot r^{2}$$

In [ ]:
A=fluopi.Area(R_frame5,Cols,T)

To improve determination of radio we can fix a monotonically incresing function to the obstained values for each colony. This function could be of sigmodial type:
\begin{equation}
A(t) = \frac{a}{1+ e^{-d(t+b)}}
\end{equation}

where $a$ is the maximum area of the colony ($a = A_{max}$) and  $b$ is the time where the colony reach half of his maximum growth ($b = t_d$)

You can se the effect of the parameters evaluating the function F_sigma:

In [ ]:
Sigmoid=fluopi.F_sigma(T, 80, -20, 0.2)
plt.plot(T,Sigmoid)

With the function Function_fit you can fit a mathematical model (F_sigma by default) to one seleceted colony:

In [ ]:
idx=Cols[2:3] # idx have to be an array, then to select one colony use this notation Cols[desired:desired+1]
init=20
end=-1
SingleF=fluopi.Function_fit(T,A,init,end,idx)

or you can perform the fit to a dictionary of selected colonies with the same function:

In [ ]:
init=21
end=-1
A_fitS=fluopi.Function_fit(T,A,init,end,Cols)

We can put alltogheter on one plot:

In [ ]:
fluopi.plot_radius(A,Cols,T)
for i in Cols:
    plt.plot(T,A_fitS[i][0],'-',label='colony '+str(i))
plt.ylabel('Area $[pixels]^2$')  #borrar estas
plt.title('Colonies Area')
#plt.xlabel('Time [h]')
plt.legend(loc='upper left')
plt.savefig("Plots/AreaFit.pdf", transparent=True)

With the area function we are able to back-compute the radius: $$ r (t) = \sqrt{\frac{A(t)}{\pi}} $$

In [ ]:
from math import pi
r_fit={}

plt.figure()
for i in Cols:
    r_fit[i]=(A_fitS[i][0]/pi)**(0.5)
    
    plt.plot(T,r_fit[i],'-',label='colony '+str(i))

plt.title('fitted radius')
plt.ylabel('radius [pixels]')
plt.xlabel('time [h]')
plt.legend(loc='upper left')
plt.savefig("Plots/colonyRadiusFit.pdf", transparent=True)

We can plot it and the blob radios together:

In [ ]:
fluopi.plot_radius(R_frame5,Cols,T)
for i in Cols:
    plt.plot(T,r_fit[i],'-',label='colony '+str(i))
plt.title('Radius')
plt.ylabel('radius [pixels]')
plt.xlabel('time [h]')
plt.legend(loc='upper left')
plt.savefig("Plots/colonyRadiusFit_points.pdf", transparent=True)

Now we can check the radio stimation again:

In [ ]:
IDCol=Cols[2]
fluopi.checkR(r_fit[IDCol],ACRoisS,IDCol,T,filename='Plots/Chemograph_fit_'+str(IDCol))


colony grows stimation seems to get more cotinuous and improve greatly at the beginning.

### Parameter estimation from data

We now have information about sub-populations of bacteria (colonies) on the same plate. The fluorescence in each region of interest estimates the time varying fluorescent protein level in each colony. The radius (or $R^2$) approximates the size of each sub-population (colony) and so is similar to optical density (OD) in a bulk plate-reader experiment. 

Lets write a simple model of fluorescent protein expression from a single cell:

\begin{equation}
\frac{dF}{dt} = k(t) - (\mu(t) + \delta(t))F
\end{equation}

where $F$ is the cell's fluorescent protein concentration, which is produced at time-varying rate $k(t)$. The cell grows at relative rate $\mu(t)=(1/V)dV/dt$ ($V=$volume) causing dilution, but assuming a constant thickness of the colony it becomes proportional to the area: $\mu(t)=(1/A)dA/dt$ ($A=$area). 
The protein is degraded at rate $\delta(t)$.


For the proteins we are using we will assume that $\delta(t)=0$ for all times. We measure the total fluorescence intensity, which is the concentration $F$ multiplied by the total volume of the colony $A_{tot} = \pi R^2$. Hence,

\begin{equation}
I(t) = F(t)A(t) \approx F(t) \pi R^2(t)
\end{equation}

From these equations it is possible to derive:

\begin{equation}
k(t) = \frac{1}{A(t)}\frac{dI}{dt}
\end{equation}


To get an acurate analysis of the dynamics of the colonies, the module include a function to compute the average value inside the limits of the colony (defined by the radio fitting) on each time. This average value correspond to $I/A$ (see the maths)

In [ ]:
MeanInt=fluopi.CRoiMeanInt_frames(sDatST,blobD,r_fit,Cols)

In [ ]:
idx=Cols[1]
plt.plot(T,MeanInt['R'][idx],'.',color='r')
plt.plot(T,MeanInt['G'][idx],'.',color='g')
plt.plot(T,MeanInt['B'][idx],'.',color='b')
plt.legend(loc='upper right')
plt.ylabel('Mean intensity')
plt.xlabel('Time [h]')
plt.title('colony '+str(idx))

by sum the three channels we can get the total intensity per area $(I/A)_{total}$

In [ ]:
#let's sum the three channels
MeanInt3C={}
for i in Cols:
    Vaux=np.zeros((MeanInt['R'][i].shape))
    for c in channels:
        Vaux+=MeanInt[c][i]
    
    MeanInt3C[i]= Vaux

In [ ]:
plt.figure()
for i in Cols:
    plt.plot(T,MeanInt3C[i],'.',label='colony '+str(i))
    plt.title('Mean Fluorescence')
    plt.xlabel('Time [h]')
    plt.ylabel('Intensity [RGB value]')
    plt.legend(loc='upper left')
    
plt.savefig("Plots/MeanInt.pdf", transparent=True)

print('this plot contain the sum of the mean of the three channels over time for each colony')

With that is possible to compute the derivative of this value: $ d(I/A)/dt $

In [ ]:
dIAdt={}
plt.figure(figsize=(10,6))
for i in Cols:
    #delta_MeanInt3C is the intensity divided by the area = d(I/A)/dt
    delta_MeanInt3C = np.diff(MeanInt3C[i])
    delta_T=np.diff(T)
    dIAdt[i] = delta_MeanInt3C/delta_T
    plt.plot(T[1:],dIAdt[i],'-',label='colony '+str(i))
    plt.xlabel('Time (h)')
    plt.ylabel('d(I/A)/dt')
    plt.ylim([-1,11])
    plt.legend(loc='upper left')
    plt.title('Colony mean intensity derivative')

Now i should try the smooth splite to correct the estimation.
Also we should try to get the $\mu(t)=(1/A)dA/dt$
that is equal to: $$\mu = \frac{2 \cdot d}{e^{d \cdot (t+b)}+1} $$

In [ ]:
Mu_val={}
for i in Cols:
    Mu_val[i]=fluopi.F_mu(T,A_fitS[i][1][1],A_fitS[i][1][2])
    plt.plot(T,Mu_val[i],'-',label='colony '+str(i))
    plt.xlabel('Time (h)')
    plt.ylabel(r'$\mu (t)$')
    plt.xlim([0,50])
    plt.ylim([0,0.3])
    plt.legend(loc='upper right')
plt.savefig("Plots/GrowthRate.pdf", transparent=True)


## Data smoothing

In [ ]:
from scipy.interpolate import UnivariateSpline

[Univariate Spline Documentation](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.interpolate.UnivariateSpline.html)

In [ ]:
dIAdt_smooth={}
#define a color vector
colors = plt.cm.Paired(np.linspace(0, 1, len(Cols))) #Paired o jet #https://matplotlib.org/users/colormaps.html

for i, c in zip(Cols, colors):
    dIAdt_smooth[i] = UnivariateSpline(T[1:],dIAdt[i], k=4)   # k is the degree of the smoothing spline  (<=5)
    #dIAdt_smooth[i].set_smoothing_factor(55)
    plt.plot(T[:],dIAdt_smooth[i](T[:]),'-',color=c, label='colony '+str(i))
    plt.plot(T[1:],dIAdt[i],'.',color=c)
    plt.xlabel('Time (h)')
    plt.ylabel('d(I/A)/dt')
    plt.legend(loc='upper left')
    #plt.ylim([-5,20])
    
plt.savefig("Plots/d(IA)dT_smooth.pdf", transparent=True)
    

Finally, the fluorescent expresion rate constant could be represented by: $$ K_f = \frac{d(I/A)}{dt} + \mu \cdot (I/A) $$

In [ ]:
Kf={}
for i, c in zip(Cols, colors):
    Kf[i]=dIAdt_smooth[i](T[:])+Mu_val[i]*MeanInt3C[i]
    plt.plot(T[:],Kf[i],'-',label='colony '+str(i))
    plt.xlabel('Time (h)')
    plt.ylabel('k(t)')
    plt.ylim([-1,18])
    plt.legend(loc='upper left')
plt.savefig("Plots/K_dynamics.pdf", transparent=True)